In [1]:
import mygrad
import mygrad.functional as F
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = mygrad.value(X_train, "x_train", requires_grad=False)
X_test = mygrad.value(X_test, "x_test", requires_grad=False)
y_train = mygrad.value(y_train, "y_train", requires_grad=False)
y_test = mygrad.value(y_test, "y_test", requires_grad=False)

In [3]:
w = mygrad.value(np.random.randn(X.shape[1]) * 10**(-2), "w", requires_grad = True)
b = mygrad.value([0.0], "b", requires_grad = True)

In [4]:
lr = 0.01

matmul1 = F.matmul()
add1 = F.add()
loss = F.bce_loss_with_logits()
s = F.sigmoid()
    
def fit(num_epoch):     
    for i in range(num_epoch):        
        y = add1.forward(matmul1.forward(X_train, w), b)
        l = loss.forward(y, y_train)
        loss.backward()
        w.data -= lr * w.grad
        b.data -= lr * b.grad
        l.zero_grad()   
        
        print(f"Epoch {i}/{num_epoch}: Train Loss: {l.data}")

def predict(X, name):
    y = add1.forward(matmul1.forward(X, w), b)
    y_pred = mygrad.value(s.forward(y).data > 0.5, name, requires_grad=False)
    
    return y_pred

Creating Function of type <class 'mygrad.functional.function.Matmul'> with name matmul_0
Creating Function of type <class 'mygrad.functional.function.Add'> with name add_0
Creating Function of type <class 'mygrad.functional.function.BCELossWithLogits'> with name bce_loss_logit_0
Creating Function of type <class 'mygrad.functional.function.Sigmoid'> with name sigmoid_0


In [5]:
fit(100)    

Epoch 0/100: Train Loss: 0.7096748855724964
Epoch 1/100: Train Loss: 0.35609139332518686
Epoch 2/100: Train Loss: 0.15912687971816736
Epoch 3/100: Train Loss: 0.09030624784537408
Epoch 4/100: Train Loss: 0.0800618656823876
Epoch 5/100: Train Loss: 0.07330399362479706
Epoch 6/100: Train Loss: 0.06865006548926807
Epoch 7/100: Train Loss: 0.06534254390541896
Epoch 8/100: Train Loss: 0.06282454903146098
Epoch 9/100: Train Loss: 0.06078366542613366
Epoch 10/100: Train Loss: 0.05905904712714122
Epoch 11/100: Train Loss: 0.057561857576857646
Epoch 12/100: Train Loss: 0.05623773087187865
Epoch 13/100: Train Loss: 0.055050410095248976
Epoch 14/100: Train Loss: 0.05397420191818507
Epoch 15/100: Train Loss: 0.0529901509716627
Epoch 16/100: Train Loss: 0.052083890736055564
Epoch 17/100: Train Loss: 0.051244318430991906
Epoch 18/100: Train Loss: 0.05046271638316405
Epoch 19/100: Train Loss: 0.04973213988288352
Epoch 20/100: Train Loss: 0.049046977686500355
Epoch 21/100: Train Loss: 0.04840263114932

In [6]:
y_train_pred = predict(X_train, "y_train_pred")
y_test_pred = predict(X_test, "y_test_pred")

In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve, matthews_corrcoef

def print_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)    

    print(f"Confusion Matrix")
    print(f"{conf_matrix}\n")
    print(f"Accuracy: {(tp + tn)/(tp + tn + fn + fp)}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")
    print(f"AUC score: {auc(fpr, tpr)}")
    print(f"MCC score: {mcc}")
    print(f"F1-Score: {(tp)/(tp+(fp+fn)/2)}\n")
    
print_metrics(y_train.data, y_train_pred.data)
print("")
print_metrics(y_test.data, y_test_pred.data)

Confusion Matrix
[[160   2]
 [  0 264]]

Accuracy: 0.9953051643192489
Precision: 0.9924812030075187
Recall: 1.0
AUC score: 0.9938271604938271
MCC score: 0.990064820428138
F1-Score: 0.9962264150943396


Confusion Matrix
[[45  5]
 [ 0 93]]

Accuracy: 0.965034965034965
Precision: 0.9489795918367347
Recall: 1.0
AUC score: 0.9500000000000001
MCC score: 0.9241653708363353
F1-Score: 0.9738219895287958

